In [1]:
import kontrol.core
import control

tf = control.ss2tf(control.rss(50))
tf_list = kontrol.core.controlutils.tf_order_split(tf, max_order=5)

5
4
4
4
5
5
5
4
4
4
4
1
4
4
4
3
4
4
4
4
3
3
4
4
4
1


In [7]:
len(tf_list)

14

In [8]:
import numpy as np
tf_split = np.prod(tf_list)

In [9]:
tf_split.pole()

array([-0.36349965+13.91205715j, -0.36349965-13.91205715j,
       -0.80475269+13.3171312j , -0.80475269-13.3171312j ,
       -1.33191646+10.86150088j, -1.33191646-10.86150088j,
       -1.15741043+10.25911817j, -1.15741043-10.25911817j,
       -6.82908169 +6.96868107j, -6.82908169 -6.96868107j,
       -1.35426567 +7.67292725j, -1.35426567 -7.67292725j,
       -1.03909673 +6.71943951j, -1.03909673 -6.71943951j,
       -5.61362338 +0.j        , -5.21580943 +1.31920912j,
       -5.21580943 -1.31920912j, -0.53771811 +4.51099357j,
       -0.53771811 -4.51099357j, -0.39640972 +3.85454523j,
       -0.39640972 -3.85454523j, -4.02081677 +0.j        ,
       -2.65911855 +0.j        , -1.59652286 +1.51125152j,
       -1.59652286 -1.51125152j, -1.64090878 +1.47206147j,
       -1.64090878 -1.47206147j, -0.48482394 +1.68318136j,
       -0.48482394 -1.68318136j, -0.74274579 +1.40186156j,
       -0.74274579 -1.40186156j, -1.6977413  +0.j        ,
       -1.58829366 +0.06155485j, -1.58829366 -0.06155485

In [5]:
tf.pole()

array([-0.36349965+13.91205715j, -0.36349965-13.91205715j,
       -0.80475269+13.3171312j , -0.80475269-13.3171312j ,
       -1.33191646+10.86150088j, -1.33191646-10.86150088j,
       -1.15741043+10.25911817j, -1.15741043-10.25911817j,
       -6.82908169 +6.96868107j, -6.82908169 -6.96868107j,
       -1.35426567 +7.67292725j, -1.35426567 -7.67292725j,
       -1.03909673 +6.71943951j, -1.03909673 -6.71943951j,
       -5.61362346 +0.j        , -5.21580948 +1.31920909j,
       -5.21580948 -1.31920909j, -0.53771811 +4.51099357j,
       -0.53771811 -4.51099357j, -0.39640972 +3.85454523j,
       -0.39640972 -3.85454523j, -4.0208148  +0.j        ,
       -2.6591433  +0.j        , -1.59652382 +1.51125465j,
       -1.59652382 -1.51125465j, -1.64090803 +1.47205717j,
       -1.64090803 -1.47205717j, -0.48482393 +1.68318136j,
       -0.48482393 -1.68318136j, -0.74274581 +1.40186156j,
       -0.74274581 -1.40186156j, -1.6936361  +0.j        ,
       -1.58935766 +0.05549824j, -1.58935766 -0.05549824

In [6]:
np.allclose(np.log(tf(1j*2*np.pi*f)), np.log(tf_split(1j*2*np.pi*f)))

NameError: name 'f' is not defined

In [ ]:
import matplotlib.pyplot as plt
f = np.logspace(-3, 3, 100000)

plt.loglog(f, abs(tf(1j*2*np.pi*f)))
plt.loglog(f, abs(tf_split(1j*2*np.pi*f)))

In [ ]:
tf.dcgain()

In [ ]:
plt.semilogx(f, np.angle(tf(1j*2*np.pi*f)))
plt.semilogx(f, np.angle(tf_split(1j*2*np.pi*f)))

In [ ]:
import kontrol
import control
import numpy as np
import matplotlib.pyplot as plt
import kontrol.frequency_series.noise_models

# lvdt_noise = kontrol.core.frequency_series.noise_models()
# lpf = control.tf([1], [1, 1])
# hpf = 1 - lpf
omega = np.logspace(-2,2,1000)
f = omega/2/np.pi
# lvdt_noise = kontrol.core.frequency_series.noise_models.lvdt_noise(f=f)
# geophone_noise = kontrol.core.frequency_series.noise_models.geophone_noise(f=f)
tf_noise1 = control.tf([1/10, 1], [1/0.1, 1]) *10
tf_noise2 = control.tf([1/0.1, 1], [1/10, 1]) 

comp = kontrol.ComplementaryFilter(noise1=tf_noise1, noise2=tf_noise2)
comp.hinfsynthesis()
comp.f = f

plt.loglog(omega, comp.noise1)
plt.loglog(omega, comp.noise2)
plt.loglog(omega, comp.noise_super)

# comp = cm.ComplementaryFilter(noise1=lvdt_noise, noise2=geophone_noise, f=f, unit="omega")

# plt.loglog(omega, lvdt_noise)
# plt.loglog(omega, geophone_noise)
# plt.loglog(omega, abs(lpf(1j*omega)))
# plt.loglog(omega, abs(hpf(1j*omega)))

In [ ]:
comp.noise_super[400:600]

In [ ]:
new_omega = np.logspace(-3, 7, 1000000)
# plt.loglog(comp.omega, abs(comp.filter1(1j*comp.omega)))
plt.loglog(new_omega, abs(comp.filter2(1j*new_omega)))
# plt.loglog(omega, abs(tf_travis(1j*omega)))
plt.loglog(new_omega, abs((1-tf_travis)(1j*new_omega)))
# tf_travis.pole()

In [ ]:
tf_correct = control.tf([1, 2.08392466e+04, 1.58200377e+04, 2.53883414e+03, 1.16522184e+02], [1, 2.08679038e+04, 1.64005175e+04, 5.55127674e+03, 8.50146586e+02])
import kontrol.core.controlutils

kontrol.core.controlutils.check_tf_equal(tf_correct, comp[0, 0], allclose_kwargs={"rtol":1e-4})

In [ ]:
tf_travis=control.tf([1.00000000e+00, 6.96103440e+03, 5.28422768e+03, 8.48018504e+02,3.89205080e+01], [1.00000000e+00, 6.97060626e+03, 5.47811529e+03, 1.85421249e+03,2.83960156e+02])

In [ ]:
?kontrol.core.controlutils.check_tf_equal

In [ ]:
tf_correct.dcgain()

In [ ]:
tf_travis.pole()

In [ ]:
comp[0, 0].pole()

In [ ]:
import kontrol.core.complementary_filter.synthesis

h1, h2 = kontrol.core.complementary_filter.synthesis.hinfcomplementary(tf_noise1, tf_noise2)

plt.loglog(omega, abs(h1(1j*omega)))
plt.loglog(omega, abs(h2(1j*omega)))

In [ ]:
comp.noise1

In [ ]:
comp.tf_noise1

In [ ]:
comp.noise1

In [ ]:
comp[0, 0]

In [ ]:
tf_noise1 = control.tf([1/10, 1], [1/0.1, 1]) *10
tf_noise2 = control.tf([1/0.1, 1], [1/10, 1]) 

comp = cm.ComplementaryFilter(noise1=tf_noise1, noise2=tf_noise2)
comp.h2synthesis()


plt.loglog(omega, abs(tf_noise1(omega*1j)))
plt.loglog(omega, abs(tf_noise2(omega*1j)))

In [ ]:
plt.loglog(omega, lvdt_noise)
plt.loglog(omega, geophone_noise)
plt.loglog(omega, comp.noise_super)
# plt.loglog(omega, abs(lpf(1j*omega)))
# plt.loglog(omega, abs(hpf(1j*omega)))

In [ ]:
comp.noise_super

In [ ]:
?kontrol.core.frequency_series.noise_models.lvdt_noise

In [ ]:
import control
import kontrol

tf = control.tf([1], [2, 3])
kontrol_tf = kontrol.TransferFunction(tf)
n_correct = "zpk([],[0.238732414637843],0.3333333333333333,\"n\")"
kontrol_tf_foton_n = kontrol_tf.foton()
# kontrol_tf.foton = "f"
f_correct = "zpk([],[-0.238732414637843],-0.5,\"f\")"
kontrol_tf_foton_f = kontrol_tf.foton()
# kontrol_tf.foton = "s"
s_correct = "zpk([],[-1.5],-0.5,\"s\")"
kontrol_tf_foton_s = kontrol_tf.foton()


In [ ]:
kontrol_tf.foton(expression="zpk", root_location="f")

In [ ]:
kontrol_tf.pole().real > 0

In [ ]:
s = control.tf("s")
tf_unstable = (s-2) / (s-1) / (s**2-s+1)

In [ ]:
kontrol_tf = kontrol.TransferFunction(tf_unstable)
kontrol_tf.stabilize()
kontrol_tf.zero().imag

In [ ]:
tf_unstable.zero()

In [ ]:
import numpy as np
a = [1,2,3,4,5,6,7,8,9,10]
step = 7
for i in range(0,10,step):
    print(a[i:i+step])

In [ ]:
?range

In [ ]:
tf = 1/(s**2 + 10/3.14*s + 10**2)
tf.pole()

In [ ]:
wn = abs(tf.pole()[0])
q = wn/(-2*tf.pole()[0].real)
q